In [20]:
import json
import requests
import pandas as pd
from langchain.prompts import PromptTemplate
from geopy.distance import geodesic
from langchain_openai import ChatOpenAI
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.tools import tool

In [15]:
NOMINATIM_API = "https://nominatim.openstreetmap.org/search"
ROUTING_API = "https://routing.openstreetmap.de/routed-foot/route/v1/driving/"
HEADERS = {"User-Agent": "GeoSpatialRAG/1.0"}

In [12]:
@tool
def get_user_location(user_prompt: str) -> str:
    """
    Extract user location from user prompt using regex

    Args:
        user_prompt (str): User prompt
    """
    location = re.search(r"My location: (.*)", user_prompt)

    return location.group(1)

@tool
def get_distance_between_locations(location1: dict, location2: dict):
    """
    Get distance between two locations defined by their coordinates (longtitude, latitude) in meters

    Args:
        location1 (dict): Location 1
        location2 (dict): Location 2
    """
    location1 = (float(location1["lat"]), float(location1["lon"]))
    location2 = (float(location2["lat"]), float(location2["lon"]))
    
    distance = geodesic(location1, location2).meters
    
    return distance

@tool
def get_route_between_locations(location1: dict, location2: dict):
    """
    Get route between two locations defined by their coordinates (longtitude, latitude).
    Route is defined as a list of steps to take to get from location1 to location2.

    Args:
        location1 (dict): Location 1
        location2 (dict): Location 2
    """
    if location1 and location2:
        url = f"{ROUTING_API}{location1['lon']},{location1['lat']};{location2['lon']},{location2['lat']}?overview=full"
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        return response.json()
    return None

In [ ]:
tools = [get_user_location, get_distance_between_locations, get_route_between_locations]

llm = ChatOpenAI(model="gpt-3.5-turbo")
llm_with_tools = llm.bind_tools(tools)